In [16]:
# Dependencies and Setup
from census import Census
from us import states
from config import (census_key, gkey)
import us
import gmaps
import os
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sodapy import Socrata
import requests
import json

In [17]:
c = Census(census_key, year=2016)
census_data = c.acs5.get(('B01003_001E', 'B17001_002E','B19013_001E'), {'for': 'county:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df= census_df.rename(columns={'B01003_001E': 'Population',
                                      'B17001_002E': 'Poverty Count',
                                      'B19013_001E': 'Median Household Income',
                                      'state':'State',
                                     'county':'County'})
# Add in Poverty Rate (Poverty Count / Population)
census_df['Poverty Rate'] = 100 * \
    census_df['Poverty Count'].astype(
        int) / census_df['Population'].astype(int)

In [18]:
# Clean Census Data
census_df=census_df[census_df.State != '72']
census_df=census_df.reset_index()
census_df['FIPS']=census_df['State']+census_df['County']
url='https://api.census.gov/data/2018/pep/population?get=DENSITY&for=county:*&in=state:*&key='+census_key
response = requests.get(url).json()
column_names = response.pop(0)
density_df=pd.DataFrame(response,columns=column_names)
density_df['FIPS']=density_df['state']+density_df['county']
merge_df = pd.merge(census_df,density_df, on="FIPS")
merge_df= merge_df.rename(columns={'DENSITY':'Population Density'})
census_df=merge_df[['FIPS','Population','Population Density','Median Household Income','Poverty Rate']]

census_df.head()

,FIPS,Population,Population Density,Median Household Income,Poverty Rate
0,05015,27690.0,44.800928174,38145.0,17.295052
1,05017,11189.0,16.199614005,29628.0,29.064260
2,05019,22684.0,25.472316857,35595.0,20.992770
3,05021,15202.0,23.217736871,32404.0,21.970793
4,05023,25587.0,45.084706902,41717.0,15.246023


In [19]:
# Read in Medicare.gov hospital compare url: https://data.medicare.gov/resource/xubh-q36u.json
dataset='xubh-q36u'
client = Socrata('data.medicare.gov', None)
hospitals = client.get(dataset,limit=6000)
hospitals_df = pd.DataFrame(hospitals)

# Clean Hospital Data
hospitals_df=hospitals_df[['hospital_name','city','state','county_name','hospital_overall_rating']]
hospitals_df=hospitals_df[hospitals_df.hospital_overall_rating != 'Not Available']
hospitals_df=hospitals_df.reset_index()
hospitals_df=hospitals_df.drop(columns='index')

hospitals_df.head()

,hospital_name,city,state,county_name,hospital_overall_rating
0,GOODLAND REGIONAL MEDICAL CENTER,GOODLAND,KS,SHERMAN,3
1,MARIA PARHAM MEDICAL CENTER,HENDERSON,NC,VANCE,2
2,ALLEGHANY COUNTY MEMORIAL HOSPITAL,SPARTA,NC,ALLEGHANY,2
3,ADVENTIST HEALTHCARE WHITE OAK MEDICAL CENTER,SILVER SPRING,MD,MONTGOMERY,3
4,ADVENTHEALTH LAKE WALES,LAKE WALES,FL,POLK,1


In [20]:
# Define Urls for the Johns Hopkins Data
confirm_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
death_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

# Read in COVID-19 Files
confirm_df=pd.read_csv(confirm_url, error_bad_lines=False)
death_df=pd.read_csv(death_url, error_bad_lines=False)

confirm_df.head()
death_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,2,3,3,4,4,4,4,4,4,4
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,0,1,1,1,1,1,1,1,1,1
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,8,11,12,15,18,20,21,23,24,33
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,0,0,0,0,0,1,1,1,1,1


In [21]:
# Clean Confirm data Rows
confirm_df=confirm_df[confirm_df.Admin2 != 'Unassigned']
confirm_df=confirm_df.dropna()
confirm_df=confirm_df[~confirm_df['Admin2'].astype(str).str.startswith('Out of')]
confirm_df=confirm_df[confirm_df.Admin2 != 'Out of*']
confirm_df=confirm_df.reset_index()
confirm_df['FIPS']=confirm_df.FIPS.map('{0:0>5.0f}'.format)

# Clean Death Data Rows
death_df=death_df[death_df.Admin2 != 'Unassigned']
death_df=death_df.dropna()
death_df=death_df[~death_df['Admin2'].astype(str).str.startswith('Out of')]
death_df=death_df[death_df.Admin2 != 'Out of*']
death_df=death_df.reset_index()

death_df.head()

,index,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,...,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,...,0,0,0,0,0,0,0,1,1,1
1,6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,...,1,1,1,1,1,1,1,1,1,1
2,7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,...,0,0,0,0,0,0,0,0,0,0
3,8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,...,0,0,0,0,0,0,0,0,0,0
4,9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Create the disease DataFrame
disease_df=confirm_df[['FIPS','Admin2','Province_State','Lat','Long_']]
disease_df= disease_df.rename(columns={'Admin2': 'County',
                                      'Province_State': 'State',
                                      'Lat': 'Latitude',
                                      'Long_':'Longitude'})
disease_df
confirm_5=[]
confirm_10=[]
confirm_20=[]
confirm_50=[]
death_5=[]
death_10=[]
death_20=[]
death_50=[]
[r,c]=confirm_df.shape

for j in range (0,r):
    for i in range (12,c):
        if confirm_df.iloc[j, i]>0:
            break
    try:
        response=confirm_df.iloc[j, i+4]
        confirm_5.append(response)
    except:
        confirm_5.append(np.nan)
    try:
        response=confirm_df.iloc[j, i+9]
        confirm_10.append(response)
    except:
        confirm_10.append(np.nan)
    try:
        response=confirm_df.iloc[j, i+19]
        confirm_20.append(response)
    except:
        confirm_20.append(np.nan)
    try:
        response=confirm_df.iloc[j, i+49]
        confirm_50.append(response)
    except:
        confirm_50.append(np.nan)
    try:
        response=death_df.iloc[j, i+5]
        death_5.append(response)
    except:
        death_5.append(np.nan)
    try:
        response=death_df.iloc[j, i+10]
        death_10.append(response)
    except:
        death_10.append(np.nan)
    try:
        response=death_df.iloc[j, i+20]
        death_20.append(response)
    except:
        death_20.append(np.nan)
    try:
        response=death_df.iloc[j, i+50]
        death_50.append(response)
    except:
        death_50.append(np.nan)
disease_df['confirm_5']=confirm_5
disease_df['confirm_10']=confirm_10
disease_df['confirm_20']=confirm_20
disease_df['confirm_50']=confirm_50
disease_df['death_5']=death_5
disease_df['death_10']=death_10
disease_df['death_20']=death_20
disease_df['death_50']=death_50

disease_df.head()

,FIPS,County,State,Latitude,Longitude,confirm_5,confirm_10,confirm_20,confirm_50,death_5,death_10,death_20,death_50
0,01001,Autauga,Alabama,32.539527,-86.644082,6.0,10.0,NaN,NaN,0.0,0.0,NaN,NaN
1,01003,Baldwin,Alabama,30.727750,-87.722071,1.0,4.0,28.0,NaN,0.0,0.0,1.0,NaN
2,01005,Barbour,Alabama,31.868263,-85.387129,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,01007,Bibb,Alabama,32.996421,-87.125115,4.0,9.0,NaN,NaN,0.0,0.0,NaN,NaN
4,01009,Blount,Alabama,33.982109,-86.567906,5.0,9.0,NaN,NaN,0.0,0.0,NaN,NaN


In [25]:
#Merge County Census Data with County COVID data
merge_df = pd.merge(disease_df, census_df, on="FIPS")
merge_df.to_csv('counties.csv',index=False,header=True)
hospitals_df.to_csv('hospitals.csv',index=False,header=True)

merge_df

,FIPS,County,State,Latitude,Longitude,confirm_5,confirm_10,confirm_20,confirm_50,death_5,death_10,death_20,death_50,Population,Population Density,Median Household Income,Poverty Rate
0,01001,Autauga,Alabama,32.539527,-86.644082,6.0,10.0,NaN,NaN,0.0,0.0,NaN,NaN,55049.0,93.534505205,53099.0,12.165525
1,01003,Baldwin,Alabama,30.727750,-87.722071,1.0,4.0,28.0,NaN,0.0,0.0,1.0,NaN,199510.0,137.13854588,51365.0,12.806877
2,01005,Barbour,Alabama,31.868263,-85.387129,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,26614.0,28.114059594,33956.0,23.427519
3,01007,Bibb,Alabama,32.996421,-87.125115,4.0,9.0,NaN,NaN,0.0,0.0,NaN,NaN,22572.0,35.986168859,39776.0,15.018607
4,01009,Blount,Alabama,33.982109,-86.567906,5.0,9.0,NaN,NaN,0.0,0.0,NaN,NaN,57704.0,89.697955279,46212.0,16.361084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Sweetwater,Wyoming,41.659439,-108.882788,1.0,3.0,NaN,NaN,0.0,0.0,NaN,NaN,44812.0,4.1288079711,68233.0,10.863162
3138,56039,Teton,Wyoming,43.935225,-110.589080,2.0,13.0,41.0,NaN,0.0,0.0,0.0,NaN,22623.0,5.7748028059,75594.0,7.293462
3139,56041,Uinta,Wyoming,41.287818,-110.547578,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,20893.0,9.7510677126,53323.0,16.570143
3140,56043,Washakie,Wyoming,43.904516,-107.680187,1.0,4.0,NaN,NaN,0.0,0.0,NaN,NaN,8351.0,3.5221743077,46212.0,14.345587
